In [1]:
import cv2
import numpy as np
import sys
import os
import math
import matplotlib.pyplot as plt

In [2]:
card = cv2.imread('./card.jpg')
text = cv2.imread('./text.jpg')
#cv2.imshow('Card image', card)
#cv2.imshow('Text image', text)
#cv2.waitKey(0)

In [59]:
text = cv2.imread('./text.jpg')
copy = text.copy()

ref_gray = cv2.cvtColor(copy, cv2.COLOR_BGR2GRAY)
thresh = 145
im_bw = cv2.threshold(ref_gray, thresh, 255, cv2.THRESH_BINARY)[1]

#kernel_size = 5
#blur_gray = cv2.GaussianBlur(ref_gray,(kernel_size, kernel_size),0)

kernel = np.ones((4,4), np.uint8)

erosion = cv2.erode(im_bw,kernel,iterations = 1)

kernel = np.ones((3,3), np.uint8)
edges = cv2.dilate(erosion, kernel, iterations=1)

closing = cv2.morphologyEx(im_bw, cv2.MORPH_CLOSE, kernel)

im2, contours, hierarchy = cv2.findContours(erosion.copy(), cv2.RETR_LIST, \
                                                cv2.CHAIN_APPROX_SIMPLE )

for contour in contours:
    area = cv2.contourArea(contour)
    text_area = text.shape[0] * text.shape[1]
    if 0.0001< area/float(text_area) < 0.002:
        (x,y,w,h) = cv2.boundingRect(contour)
        cv2.rectangle(text, (x,y), (x+w,y+h), (0,255,0), 2)

#v2.imshow('thresh', im_bw)
cv2.imshow('Text', text)
#v2.imshow('closing', closing)
#v2.imshow('erosion', erosion)
#v2.imshow('edges', edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [60]:
cv2.imwrite('out.jpg', text)

True

In [15]:
card = cv2.imread('./cell_pic.jpg')
orig = card.copy()

rows, cols = card.shape[:2]
card = cv2.resize(card,(int(cols*0.5), int(rows*0.5)), interpolation = cv2.INTER_AREA)

gray = cv2.cvtColor(card,cv2.COLOR_BGR2GRAY)

kernel_size = 5
blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)

low_threshold = 50
high_threshold = 150
edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

cv2.imshow('line', card)
cv2.imshow('line 1', edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
im2, contours, hierarchy = cv2.findContours(edges.copy(), cv2.RETR_LIST, \
                                                cv2.CHAIN_APPROX_SIMPLE )

contours = sorted(contours, key=cv2.contourArea, reverse=True)[:1]
perimeter = cv2.arcLength(contours[0], True) 
ROIdimensions = cv2.approxPolyDP(contours[0], 0.02*perimeter, True)

cv2.drawContours(card, [ROIdimensions], -1, (0,255,0), 2)
cv2.imshow('contour', card)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [17]:
# reshape coordinates array
ROIdimensions = ROIdimensions.reshape(4,2)

# list to hold ROI coordinates
rect = np.zeros((4,2), dtype='float32')
                
# top left corner will have the smallest sum, 
# bottom right corner will have the largest sum
                
s = np.sum(ROIdimensions, axis=1)
rect[0] = ROIdimensions[np.argmin(s)]
rect[2] = ROIdimensions[np.argmax(s)]
                
# top-right will have smallest difference
# botton left will have largest difference
diff = np.diff(ROIdimensions, axis=1)
rect[1] = ROIdimensions[np.argmin(diff)]
rect[3] = ROIdimensions[np.argmax(diff)]
                
# top-left, top-right, bottom-right, bottom-left
(tl, tr, br, bl) = rect
                
# compute width of ROI
widthA = np.sqrt((tl[0] - tr[0])**2 + (tl[1] - tr[1])**2 )
widthB = np.sqrt((bl[0] - br[0])**2 + (bl[1] - br[1])**2 )
maxWidth = max(int(widthA), int(widthB))
                
# compute height of ROI
heightA = np.sqrt((tl[0] - bl[0])**2 + (tl[1] - bl[1])**2 )
heightB = np.sqrt((tr[0] - br[0])**2 + (tr[1] - br[1])**2 )
maxHeight = max(int(heightA), int(heightB))

In [18]:
# Set of destinations points for "birds eye view"
# dimension of the new image
dst = np.array([
    [0,0],
    [maxWidth-1, 0],
    [maxWidth-1, maxHeight-1],
    [0, maxHeight-1]], dtype="float32")

# compute the perspective transform matrix and then apply it
transformMatrix = cv2.getPerspectiveTransform(rect, dst)

# transform ROI
scan = cv2.warpPerspective(card, transformMatrix, (maxWidth, maxHeight))

# lets see the wraped document
cv2.imshow("Scaned",scan)
cv2.waitKey(0)
cv2.destroyAllWindows()